In [1]:
import datasets
import numpy as np
import cleaning
from get_data import fetch_data
import re
num_proc = 8

/home/ubuntu/miniconda3/envs/HW3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data, val_data = fetch_data()

/home/ubuntu/miniconda3/envs/HW3/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [3]:
# Filtering non-english text at word level
non_english_regex = re.compile(r'[^\u0000-\u007F]+')
def filter_non_english(example):
    text = example['text']
    words = text.split()
    english_words = [w for w in words if not cleaning.filter_noneng(w)]
    return {'text': " ".join(english_words), 'before_filter': len(words), 'after_filter': len(english_words)}

In [4]:
def filter_pii(example):
    filtered_text = cleaning.clean_pii(example['text'])
    return {'text': filtered_text, 'before_filter': len(example['text']), 'after_filter': len(filtered_text)}

In [5]:
def clean(dataset):
    print("Filtering non-english words: ")
    dataset = dataset.map(filter_non_english, num_proc = num_proc)
    before_filter = np.sum(dataset['before_filter'])
    after_filter = np.sum(dataset['after_filter'])
    print(f'Words before filtering: {before_filter}')
    print(f'Words after filtering: {after_filter}')
    print(f'Words removed: {before_filter - after_filter}')
    print()

    print("Applying Offensive Words filtering: ")
    before_filter = len(dataset)
    bad_words_list = open('data/bad_words_list.txt').read().split('\n')
    dataset = dataset.filter(lambda x: cleaning.clean_other(x['text'], bad_words=bad_words_list), num_proc=num_proc)
    after_filter = len(dataset)
    print(f'Length before filtering: {before_filter}')
    print(f'Length after filtering: {after_filter}')
    print(f'Documents removed: {before_filter - after_filter}')
    print()

    print("Applying PII filter: ")
    dataset = dataset.map(filter_pii, num_proc=8)
    before_filter = np.sum(dataset['before_filter'])
    after_filter = np.sum(dataset['after_filter'])
    print(f"Charaters before filtering: {before_filter}")
    print(f"Charaters after filtering: {after_filter}")
    print(f'Characters removed: {before_filter - after_filter}')
    
    return dataset

In [6]:
train_data_filtered = clean(train_data)

Filtering non-english words: 
Words before filtering: 264841705
Words after filtering: 258956021
Words removed: 5885684

Applying Offensive Words filtering: 
Length before filtering: 281979
Length after filtering: 250435
Documents removed: 31544

Applying PII filter: 
Charaters before filtering: 1285062338
Charaters after filtering: 1278348826
Characters removed: 6713512


In [7]:
print(len(train_data_filtered))

250435


In [8]:
val_data_filtered = clean(val_data)

Filtering non-english words: 
Words before filtering: 13952369
Words after filtering: 13608836
Words removed: 343533

Applying Offensive Words filtering: 
Length before filtering: 14842
Length after filtering: 13225
Documents removed: 1617

Applying PII filter: 
Charaters before filtering: 67692417
Charaters after filtering: 67353887
Characters removed: 338530


In [9]:
cols_to_remove = ['before_filter', 'after_filter']
filtered_dataset = datasets.DatasetDict(
    {
        'train': train_data_filtered.remove_columns(cols_to_remove), 
        'val': val_data_filtered .remove_columns(cols_to_remove)
    }
)
filtered_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'timestamp', 'url'],
        num_rows: 250435
    })
    val: Dataset({
        features: ['text', 'timestamp', 'url'],
        num_rows: 13225
    })
})

In [14]:
filtered_dataset.save_to_disk('cleaned_data.arrow')

Saving the dataset (1/1 shards): 100%|██████████| 13225/13225 [00:00<00:00, 263694.26 examples/s]
